In [19]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [20]:
data_path = "/home/bhavik/Desktop/Mask AI/Face-Mask-Detection/dataset"
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories, labels))

In [21]:
img_size = (160, 160)
data = []
target = []

for category in categories:
    folder_path = os.path.join(data_path, category)
    img_names = os.listdir(folder_path)

    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        try:
            resized = cv2.resize(img, img_size)

            data.append(resized)
            target.append(label_dict[category])

        except Exception as e:
            print("Exception: ", e)

data = np.array(data)/255.0
data = np.reshape(data, (data.shape[0], img_size[0], img_size[1], 3))
target = np.array(target)
target = to_categorical(target)

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, shuffle = True)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, shuffle = True)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/bhavik/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_32081/2822627093.py", line 11, in <cell line: 5>
    img = cv2.imread(img_path)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/bhavik/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/bhavik/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/bhavik/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1012, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/home/bhavik/.local/lib/python3.8/site-packages/IPython/core/ultratb.py

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [ ]:
vgg = VGG16(input_shape = (160, 160, 3), weights = 'imagenet', include_top = False)

for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)

prediction = Dense(units = 2, activation = 'softmax')(Dense(units = 64, activation = 'relu')(x))

model = Model(inputs = vgg.input, outputs = prediction)

print(model.summary())

2022-12-24 23:13:33.877404: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bhavik/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-12-24 23:13:33.877538: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-24 23:13:33.877591: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhavik-HP-Pavilion-x360-Convertible-14-cd0xxx): /proc/driver/nvidia/version does not exist
2022-12-24 23:13:33.878226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler f

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 160, 160, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 160, 160, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 80, 80, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 80, 80, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 80, 80, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 40, 40, 128)       0     

In [ ]:
datagen = ImageDataGenerator(
    rotation_range = 15,
    zoom_range = 0.15,
    width_shift_range = 0.1,
    height_shift_range = 0.1
)

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer= 'adam', metrics = ['accuracy'])

history = model.fit(datagen.flow(X_train, y_train, batch_size = 16), epochs = 5, validation_data = (X_val, y_val))

Epoch 1/5
192/192 [==============================] - 863s 4s/step - loss: 0.1776 - accuracy: 0.9331 - val_loss: 0.1251 - val_accuracy: 0.9556
Epoch 2/5
192/192 [==============================] - 696s 4s/step - loss: 0.0675 - accuracy: 0.9749 - val_loss: 0.1405 - val_accuracy: 0.9452
Epoch 3/5
192/192 [==============================] - 695s 4s/step - loss: 0.0610 - accuracy: 0.9785 - val_loss: 0.1333 - val_accuracy: 0.9634
Epoch 4/5
192/192 [==============================] - 708s 4s/step - loss: 0.0331 - accuracy: 0.9902 - val_loss: 0.1103 - val_accuracy: 0.9765
Epoch 5/5
192/192 [==============================] - 775s 4s/step - loss: 0.0397 - accuracy: 0.9876 - val_loss: 0.1101 - val_accuracy: 0.9791


In [22]:
model.save("Model.h5")

In [2]:
import cv2
import time
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
maskClassifier = load_model('/home/bhavik/Desktop/Mask AI/Model.h5')

while True:
   
    _, frame = cap.read()

    if not _:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        )
    
    for (x, y, h, w) in faces:

        faceROI = frame[y : y + h, x : x + w, :]

        faceROI = cv2.resize(faceROI, (160, 160))
        faceROI = img_to_array(faceROI)
        faceROI = faceROI.reshape(1, 160, 160, 3)

        prediction = maskClassifier(faceROI)
        (withoutmask, withmask) = prediction[0].numpy()

        (label, color, prob) = ('Mask', (0, 255, 0), withmask*100.0) if withmask > withoutmask else ('No mask', (0, 0, 255), withoutmask*100.0)

        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

        cv2.rectangle(frame, (x + 15, y + 2), (x + w - 15, y + 20), (0, 0, 0), -1) #lower
        cv2.rectangle(frame, (x + 15, y - 2), (x + w - 15, y - 20), (0, 0, 0), -1) #upper

        cv2.putText(frame, str(prob)+' %', (x + 20, y - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.putText(frame, label, (x + 20, y + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)

        
    cv2.imshow('Video', frame)
    

    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()